In [78]:
import pandas as pd

In [79]:
df = pd.read_excel("/home/claudio/Downloads/Encuesta_limpieza_22_02_23 vEFRL sinmenores.xlsx", index_col=0)

In [80]:
df.shape

(1803, 101)

In [81]:
df['Provincia donde vive'].value_counts()

Ica        1077
Pisco       376
Chincha     345
Nazca         5
Name: Provincia donde vive, dtype: int64

In [82]:
df['Provincia donde vive'].value_counts(normalize=True)

Ica        0.597338
Pisco      0.208541
Chincha    0.191348
Nazca      0.002773
Name: Provincia donde vive, dtype: float64

In [83]:
miembros_familia = pd.read_excel("inputs/Encuesta_para_inmigrantes_venezolanos_en_Ica_-_all_versions_-_labels_-_2023-02-18-20-40-30.xlsx",
             sheet_name=1)

In [84]:
miembros_familia = miembros_familia.rename(
    columns={'Indique la edad del miembro de familia': 'edad'})
miembros_familia['grupo_edad_INEI'] = pd.cut(
    x=miembros_familia['edad'], bins=[0,14,59,100],
    labels=['0-14', '15-59', '60+'])

In [85]:
miembros_familia_edad = (miembros_familia
    .groupby(by=['_parent_index', 'grupo_edad_INEI'], as_index=False)
    .size()
    .pivot(index='_parent_index', columns='grupo_edad_INEI', values="size")
    .fillna(0))
miembros_familia_edad.columns = [f"n_miembros_{values}" for values in miembros_familia_edad.columns]    

In [88]:
df = (df
    .merge(
        right=miembros_familia_edad, how='left', 
        left_index=True, right_on='_parent_index')
    .drop(labels="_parent_index", axis=1))
df.loc[:, miembros_familia_edad.columns[1:]] = df[miembros_familia_edad.columns[1:]].fillna(0.0)

In [ ]:
tabla_edad = (df.groupby(['¿Cuál es SU sexo?', 'Provincia donde vive'])
.agg({'n_miembros_0-14': 'sum',
      'n_miembros_15-59': 'sum',
      'n_miembros_60+': 'sum'})
 .unstack()
 .swaplevel(axis=1)
 .sort_index(axis=1))
total = tabla_edad.sum(axis=0)
total.name = 'Total'
tabla_edad.append(total)

In [227]:
tabla_edad

Provincia donde vive         Chincha                                  \
                     n_miembros_0-14 n_miembros_15-59 n_miembros_60+   
¿Cuál es SU sexo?                                                      
Femenino                       431.0            369.0           23.0   
Masculino                       61.0             51.0            4.0   

Provincia donde vive             Ica                                  \
                     n_miembros_0-14 n_miembros_15-59 n_miembros_60+   
¿Cuál es SU sexo?                                                      
Femenino                      1365.0           1127.0           72.0   
Masculino                      207.0            146.0            5.0   

Provincia donde vive           Nazca                                  \
                     n_miembros_0-14 n_miembros_15-59 n_miembros_60+   
¿Cuál es SU sexo?                                                      
Femenino                         7.0              6.0            0.0   
Masculino                        0.0              1.0            0.0   

Provincia donde vive           Pisco                                  
                     n_miembros_0-14 n_miembros_15-59 n_miembros_60+  
¿Cuál es SU sexo?                                                     
Femenino                       499.0            429.0           36.0  
Masculino                       31.0             36.0            1.0

In [228]:
tabla_edad.to_clipboard()

In [110]:
df['¿Cuál es SU edad?'].min()

18.0

In [113]:
df['¿Cuál es SU edad?'].isna().sum()

1

In [131]:
df['n_miembros_menores'].max()

9

In [133]:
df['n_miembros_menores_bin'] = pd.cut(df['n_miembros_menores'], bins=[0.5, 1, 4, 10], labels=['1','2-3','4+'])

In [134]:
un_adulto_cargo = df[(df['n_miembros_menores'] > 0) & \
(df['n_miembros_adultos'] == 0) & \
(df['n_miembros_mayores'] == 0)]

In [128]:
un_adulto_cargo.shape

(370, 104)

In [229]:
tabla_adulto_cargo = un_adulto_cargo.groupby(['Provincia donde vive', 'n_miembros_menores_bin']).size().unstack(0)

In [230]:
tabla_adulto_cargo

Provincia donde vive,Chincha,Ica,Pisco
n_miembros_menores_bin,,,
1,25,74,25
2-3,49,133,58
4+,2,3,1


In [231]:
tabla_adulto_cargo.to_clipboard()

In [ ]:
df['¿Cuál es SU actual ESTATUS MIGRATORIO?'].unique()

array(['Mantengo mi documento de identidad venezolano', 'CPP',
       'Carnet de Extranjeria', 'No posee documentos',
       'Documento de identidad peruano', 'No especificado',
       'Carnet de Solicitud de Refugio'], dtype=object)

In [153]:
df['Situación migratoria'] = df['¿Cuál es SU actual ESTATUS MIGRATORIO?'].replace({'Mantengo mi documento de identidad venezolano':'Cédula Venezolana',
'Documento de identidad peruano':'Otro',
'Carnet de Solicitud de Refugio':'Otro', 
'No especificado':'Otro',
'No posee documentos':'Otro'})

In [232]:
sit_mig = df.groupby(['Provincia donde vive', 'Situación migratoria']).size().unstack(0).fillna(0)
sit_mig

Provincia donde vive,Chincha,Ica,Nazca,Pisco
Situación migratoria,,,,
CPP,128.0,391.0,2.0,109.0
Carnet de Extranjeria,74.0,207.0,1.0,68.0
Cédula Venezolana,125.0,426.0,2.0,172.0
Otro,18.0,53.0,0.0,27.0


In [233]:
sit_mig.to_clipboard()

In [160]:
df['Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?'].unique()

array(['NO', 'No tengo trabajo', 'No especifica', 'SÍ', 'Sin respuesta'],
      dtype=object)

In [234]:
contrato_formal = df.groupby(['Provincia donde vive', 'Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?']).size().unstack(0).fillna(0)
contrato_formal

Provincia donde vive,Chincha,Ica,Nazca,Pisco
"Actualmente, ¿USTED cuenta con un CONTRATO DE TRABAJO FORMAL?",,,,
NO,297.0,993.0,5.0,360.0
No especifica,4.0,3.0,0.0,0.0
No tengo trabajo,24.0,54.0,0.0,12.0
Sin respuesta,1.0,1.0,0.0,0.0
SÍ,19.0,26.0,0.0,4.0


In [235]:
contrato_formal.to_clipboard()

In [236]:
rel_alim = (df.groupby(['Provincia donde vive', 'Actualmente ¿SU TRABAJO SE ENCUENTRA VINCULADO A TEMAS ALIMENTARIOS?'])
.size()
.unstack(0)
.fillna(0))
rel_alim

Provincia donde vive,Chincha,Ica,Nazca,Pisco
Actualmente ¿SU TRABAJO SE ENCUENTRA VINCULADO A TEMAS ALIMENTARIOS?,,,,
NO,207.0,671.0,4.0,208.0
No aplica,24.0,54.0,0.0,12.0
Sin respuesta,13.0,2.0,0.0,0.0
SÍ,101.0,350.0,1.0,156.0


In [237]:
rel_alim.to_clipboard()

In [238]:
rel_alim_detalle = (df.groupby(['Provincia donde vive', '¿Podría decirnos a que se dedica?'])
.size()
.unstack(0)
.fillna(0))
rel_alim_detalle

Provincia donde vive,Chincha,Ica,Nazca,Pisco
¿Podría decirnos a que se dedica?,,,,
Cargador en mercado/estibador,1.0,0.0,0.0,0.0
Delivery,1.0,3.0,0.0,0.0
"Empleados en actividades alimentarias (avícolas, pescaderías, chacra, otros)",11.0,39.0,0.0,43.0
"Otro, especificar",2.0,3.0,0.0,1.0
Sin respuesta,13.0,2.0,0.0,0.0
"Trabajo en restaurante, PASTELERIA, PANADERIA (en cualquier actividad)",14.0,92.0,0.0,30.0
"Vendedor en mercado, bodega o similar (ej: vendedor de helados)",4.0,13.0,1.0,2.0
Venta ambulante de alimentos (Tipo 1).,64.0,153.0,0.0,69.0
Venta de alimentos con uno o más empleados (tipo 3),3.0,9.0,0.0,2.0


In [239]:
rel_alim_detalle.to_clipboard()

In [240]:
no_rel_alim_detalle = (df.groupby(['Provincia donde vive', '¿Podría decirnos a que se dedica?.1'])
.size()
.unstack(0)
.fillna(0))
no_rel_alim_detalle

Provincia donde vive,Chincha,Ica,Nazca,Pisco
¿Podría decirnos a que se dedica?.1,,,,
ACTIVIDADES RELACIONADAS A LA CONSTRUCCIÓN,13.0,23.0,0.0,4.0
ACTIVIDADES RELACIONADAS A LA EXTRACCIÓN,0.0,2.0,0.0,2.0
ACTIVIDADES RELACIONADAS AL SECTOR SALUD,0.0,1.0,0.0,0.0
ACTIVIDADES RELACIONADAS AL SECTOR TRANSPORTE,17.0,41.0,1.0,4.0
ALMACÉN/OBRERO,4.0,4.0,0.0,1.0
AMA/O DE CASA,100.0,374.0,2.0,132.0
ATENCIÓN AL CLIENTE,4.0,13.0,0.0,3.0
AYUDANTE SERVICIOS GENERALES,6.0,10.0,0.0,3.0
CUIDADO/ENSEÑANZA/EDUCACIÓN NIÑO/AS,5.0,17.0,0.0,4.0


In [241]:
no_rel_alim_detalle.to_clipboard()

In [243]:
emprendimiento = (df.groupby(['Provincia donde vive', 'Si tiene algun tipo de emprendimiento. ¿Podría decirnos cuál es el rubro de su emprendimiento?',])
.size()
.unstack(0)
.fillna(0))
emprendimiento

Provincia donde vive,Chincha,Ica,Pisco
Si tiene algun tipo de emprendimiento. ¿Podría decirnos cuál es el rubro de su emprendimiento?,,,
Bodega o negocio propio,0.0,1.0,0.0
Otro,16.0,26.0,18.0
Venta de alimentos,60.0,188.0,71.0
Venta de ropa,5.0,11.0,5.0


In [244]:
emprendimiento.to_clipboard()

In [198]:
df['Preocupación por no tener alimentos (Leve)'] = df['En los últimos 12 meses ¿Usted u otra persona en su hogar se haya preocupado por no tener suficientes alimentos para comer por falta de dinero u otros recursos?']
df['No pudo comer alimentos saludables y nutritivos (v)'] = df['Pensando aún en los últimos 12 meses, ¿hubo alguna vez en que usted u otra persona en su hogar no haya podido comer alimentos saludables y nutritivos  por falta de dinero u otros recursos?']
df['Comió poca variedad de alimentos (Moderada)'] = df['En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya comido poca variedad de  alimentos por falta de dinero u otros recursos?']
df['Se saltó alguna comida por falta (Moderada)'] = df['En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya tenido que dejar de desayunar, almorzar o cenar porque no había suficiente dinero u otros recursos para obtener alimentos?']
df['Comió menos de lo que pensaba que debía (Moderada)'] = df['En los últimos 12 meses Pensando aún en los últimos 12 meses, ¿hubo alguna vez en que usted u otra persona en su hogar haya comido menos de lo que pensaba que debía comer por falta de dinero u otros recursos?']
df['Se quedó sin alimentos (Grave)'] = df['En los últimos 12 meses ¿Hubo alguna vez en que su hogar se haya quedado sin alimentos por falta de dinero u otros recursos?']
df['Tuvo hambre pero no comió por falta de alientos (Moderada)'] = df['En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya sentido  hambre pero no comió porque no había suficiente dinero u otros recursos para obtener alimentos?']
df['Dejo de comer todo un día (Grave)'] = df['En los últimos 12 meses ¿Hubo alguna vez en que usted u otra persona en su hogar haya dejado de comer todo un día por falta de dinero u otros recursos?']

In [200]:
sit_alim  = ['Preocupación por no tener alimentos (Leve)',
'No pudo comer alimentos saludables y nutritivos (v)',
'Comió poca variedad de alimentos (Moderada)',
'Comió menos de lo que pensaba que debía (Moderada)',
'Se saltó alguna comida por falta (Moderada)',
'Tuvo hambre pero no comió por falta de alientos (Moderada)',
'Se quedó sin alimentos (Grave)',
'Dejo de comer todo un día (Grave)']


In [222]:
table = []
for col in sit_alim:
    dff = df[df[col] == 'Si']
    table_col = (dff.groupby(['Provincia donde vive'])
     .size()
     .fillna(0))
    table_col.name = col
    table.append(table_col)
table_sit_alim = pd.concat(table, axis=1).T.fillna(0)
table_sit_alim

Provincia donde vive,Chincha,Ica,Nazca,Pisco
Preocupación por no tener alimentos (Leve),328.0,1002.0,5.0,360.0
No pudo comer alimentos saludables y nutritivos (v),267.0,761.0,3.0,274.0
Comió poca variedad de alimentos (Moderada),297.0,848.0,2.0,325.0
Comió menos de lo que pensaba que debía (Moderada),284.0,773.0,3.0,304.0
Se saltó alguna comida por falta (Moderada),239.0,677.0,2.0,241.0
Tuvo hambre pero no comió por falta de alientos (Moderada),226.0,612.0,2.0,266.0
Se quedó sin alimentos (Grave),193.0,562.0,2.0,215.0
Dejo de comer todo un día (Grave),71.0,232.0,0.0,97.0


In [245]:
table_sit_alim.to_clipboard()

In [225]:
df['¿CUÁNDO USTED O ALGUN MIEMBRO DE SU HOGAR SE ENFERMA, A DONDE ACUDE PARA LA ATENCIÓN?'].unique()


array(['POSTA', 'Automedicación', 'Botica', 'HOSPITAL', nan,
       'CLÍNICA PRIVADA', 'Campañas o ferias de salud', 'ESSALUD',
       'OTRO, CUAL?', 'MEDICO PARTICULAR'], dtype=object)

In [248]:
atnncion_medica = (df.groupby(['Provincia donde vive', '¿CUÁNDO USTED O ALGUN MIEMBRO DE SU HOGAR SE ENFERMA, A DONDE ACUDE PARA LA ATENCIÓN?'])
.size()
.unstack(0)
.fillna(0))
atnncion_medica

Provincia donde vive,Chincha,Ica,Nazca,Pisco
"¿CUÁNDO USTED O ALGUN MIEMBRO DE SU HOGAR SE ENFERMA, A DONDE ACUDE PARA LA ATENCIÓN?",,,,
Automedicación,60.0,159.0,0.0,74.0
Botica,36.0,141.0,1.0,30.0
CLÍNICA PRIVADA,5.0,4.0,0.0,0.0
Campañas o ferias de salud,1.0,1.0,0.0,0.0
ESSALUD,1.0,5.0,0.0,0.0
HOSPITAL,99.0,184.0,4.0,89.0
MEDICO PARTICULAR,1.0,4.0,0.0,1.0
"OTRO, CUAL?",1.0,8.0,0.0,2.0
POSTA,136.0,570.0,0.0,180.0
